### **Memory** 

- Save your conversation with the LLM 

**Intro**

- Ability to store information about past interactions 
- **Most of memory-related functionality in LangChain is marked as beta**. This is for two reasons:
      1. Most functionality are not production ready.
      2. Most functionality work with Legacy chains, not the newer LCEL sytax.
- **The main exception to this is the ChatMessageHistory functionality**. This functionality is largely production ready and does integrate with LCEL

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

**Buffer Memory**

- ConversationBufferMemory keep a list of chat messages in a buffer and pases those into the prompt template.

In [4]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain 
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The 'variable_name' here is what must align with memory 
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

C:\Users\HP\AppData\Local\Temp\ipykernel_16068\760406290.py:23: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  conversation = LLMChain(


In [5]:
conversation({"question": "hi"})

C:\Users\HP\AppData\Local\Temp\ipykernel_16068\2596321432.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  conversation({"question": "hi"})




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?')],
 'text': 'Hello! How are you today?'}

In [7]:
conversation({"question": "My name is Haseebullah and i have moved 33 times. "})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Haseebullah and i have moved 33 times. 
AI: Nice to meet you, Haseebullah! Moving 33 times must have been quite an adventure. What made you move so many times?
Human: My name is Haseebullah and i have moved 33 times. 

> Finished chain.


{'question': 'My name is Haseebullah and i have moved 33 times. ',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='My name is Haseebullah and i have moved 33 times. '),
  AIMessage(content='Nice to meet you, Haseebullah! Moving 33 times must have been quite an adventure. What made you move so many times?'),
  HumanMessage(content='My name is Haseebullah and i have moved 33 times. '),
  AIMessage(content="It seems like you have a unique experience of moving around so much. Have you enjoyed the variety of places you've lived in, or has it been challenging for you?")],
 'text': "It seems like you have a unique experience of moving around so much. Have you enjoyed the variety of places you've lived in, or has it been challenging for you?"}

In [8]:
conversation({"question": "Do you remember my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi
AI: Hello! How are you today?
Human: My name is Haseebullah and i have moved 33 times. 
AI: Nice to meet you, Haseebullah! Moving 33 times must have been quite an adventure. What made you move so many times?
Human: My name is Haseebullah and i have moved 33 times. 
AI: It seems like you have a unique experience of moving around so much. Have you enjoyed the variety of places you've lived in, or has it been challenging for you?
Human: Do you remember my name?

> Finished chain.


{'question': 'Do you remember my name?',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How are you today?'),
  HumanMessage(content='My name is Haseebullah and i have moved 33 times. '),
  AIMessage(content='Nice to meet you, Haseebullah! Moving 33 times must have been quite an adventure. What made you move so many times?'),
  HumanMessage(content='My name is Haseebullah and i have moved 33 times. '),
  AIMessage(content="It seems like you have a unique experience of moving around so much. Have you enjoyed the variety of places you've lived in, or has it been challenging for you?"),
  HumanMessage(content='Do you remember my name?'),
  AIMessage(content="Yes, your name is Haseebullah. It's nice to chat with you, Haseebullah!")],
 'text': "Yes, your name is Haseebullah. It's nice to chat with you, Haseebullah!"}

In [9]:
print(memory.buffer)

[HumanMessage(content='hi'), AIMessage(content='Hello! How are you today?'), HumanMessage(content='My name is Haseebullah and i have moved 33 times. '), AIMessage(content='Nice to meet you, Haseebullah! Moving 33 times must have been quite an adventure. What made you move so many times?'), HumanMessage(content='My name is Haseebullah and i have moved 33 times. '), AIMessage(content="It seems like you have a unique experience of moving around so much. Have you enjoyed the variety of places you've lived in, or has it been challenging for you?"), HumanMessage(content='Do you remember my name?'), AIMessage(content="Yes, your name is Haseebullah. It's nice to chat with you, Haseebullah!")]


### **Conversation Buffer Window** 
Similar to the previous one, but you can limit the number of conversational exchanges stored in memory. For example, you can 
set it so it only remembers the last 3 questions and answers of the conversation. 

In [10]:
from langchain.memory import ConversationBufferWindowMemory 
from langchain.chains import ConversationChain


In [21]:
window_memory = ConversationBufferMemory(k=3)

In [22]:
conversation_window = ConversationChain(
    llm=llm,
    memory = window_memory,
    verbose= True,
    
)

In [23]:
conversation_window({"input:" "Hi, my name is Haseeb"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: {'input:Hi, my name is Haseeb'}
AI:

> Finished chain.


{'input': {'input:Hi, my name is Haseeb'},
 'history': '',
 'response': "Hello Haseeb! It's nice to meet you. How can I assist you today?"}

In [26]:
conversation_window({"input": "My favorite color is blue"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI:

> Finished chain.


{'input': 'My favorite color is blue',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?",
 'response': "Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?"}

In [27]:
conversation_window({"input": "My favourite animal is cat."})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI:

> Finished chain.


{'input': 'My favourite animal is cat.',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?",
 'response': 'Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?'}

In [28]:
conversation_window({"input": "I like to drive a vespa scooter in the city"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'I like to drive a vespa scooter in the city',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?",
 'response': 'Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?'}

In [29]:
conversation_window({"input": "My favorite city is New York"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'My favorite city is New York',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?",
 'r

In [30]:
conversation_window({"input": "what is my favorite my color?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'what is my favorite my color?',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?\nHum

In [31]:
conversation_window({"input": "what is my favorite city?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'what is my favorite city?',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?\nHuman: 

In [32]:
conversation_window({"input": "which is my favorite animal?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'which is my favorite animal?',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?\nHuma

In [33]:
conversation_window({"input": "what is my name?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: ['input:Hi, my name is Haseeb']
AI: Hello Haseeb! It's nice to meet you. How can I assist you today?
Human: what is my favorite my color?
AI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?
Human: what is my favorite city?
AI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?
Human: My favorite color is blue
AI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?
Human: My favourite animal is cat.
AI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have 

{'input': 'what is my name?',
 'history': "Human: ['input:Hi, my name is Haseeb']\nAI: Hello Haseeb! It's nice to meet you. How can I assist you today?\nHuman: what is my favorite my color?\nAI: I'm sorry, Haseeb, I do not have that information. What is your favorite color?\nHuman: what is my favorite city?\nAI: I'm sorry, Haseeb, I do not have that information either. Could you please let me know what your favorite city is?\nHuman: My favorite color is blue\nAI: Blue is a great choice! It's a calming and serene color. Is there anything else you'd like to know or talk about, Haseeb?\nHuman: My favourite animal is cat.\nAI: Cats are wonderful animals! They are known for their independence and playful nature. Do you have a cat as a pet, Haseeb?\nHuman: I like to drive a vespa scooter in the city\nAI: Vespa scooters are a stylish and fun way to get around the city! They are known for their retro design and smooth ride. Have you ever ridden a Vespa scooter before, Haseeb?\nHuman: My favori

### **Chat Message History**

- Save list of chat messages and then fetch them all 

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [35]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message("hi!")

history.add_ai_message("whats up?")

In [36]:
history.messages 

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]